# Single level adaptive sampling

## Example setup

We'll create an experimental design for the following toy simulator function on a
2-dimensional input space.

In [1]:
import numpy as np
from exauq.core.modelling import SimulatorDomain


domain = SimulatorDomain([(0, 1), (0, 1)])

def simulation_func(x: np.ndarray) -> float:
    return x[1] + x[0]**2 + x[1]**2 - np.sqrt(2) + np.sin(2 * np.pi * x[0]) + np.sin(4 * np.pi * x[0] * x[1])


Make some randomly-generated simulator inputs and calculated outputs, as an initial design.
We'll use a Latin hypercube sampling, as provided by [scipy](https://scipy.org/).

In [44]:
from scipy.stats.qmc import LatinHypercube
from exauq.core.modelling import TrainingDatum


sampler = LatinHypercube(domain.dim)
initial_design_arr = sampler.random(n=10)  # will sample in the unit cube, which is what we want in this case
initial_outputs = np.array([simulation_func(x) for x in initial_design_arr])
initial_data = TrainingDatum.list_from_arrays(initial_design_arr, initial_outputs)

## Set up GP with initial design

Next we'll train a GP. We first create a new GP using the `MogpEmulator` class, specifying that it uses a Matern 5/2 kernel.

In [45]:
from exauq.core.emulators import MogpEmulator

# Don't display warnings from mogp-emulator
import warnings
warnings.filterwarnings("ignore")

gp = MogpEmulator(kernel="Matern52")
gp.fit(initial_data)

Too few unique inputs; defaulting to flat priors
Too few unique inputs; defaulting to flat priors


## Find new design points using leave-one-out adaptive sampling

Let's now find a new design point using the leave-one-out adaptive design methodology. We
use the function `compute_single_level_loo_samples` to do this. This function requires two
arguments:
- The GP to find the new design point for.
- The `SimulatorDomain` object defining the input space on which the simulator inputs are
  defined.

By default, a single, new design point will be calculated:

In [46]:
from exauq.core.designers import compute_single_level_loo_samples

new_design_pts = compute_single_level_loo_samples(gp, domain)
new_design_pts[0]

Input(np.float64(0.4958093479799048), np.float64(7.637374342395198e-07))

If instead we want to compute a batch of training inputs in one go, we can do this by
specifying a different batch size:

In [47]:
new_design_pts = compute_single_level_loo_samples(gp, domain, batch_size=5)
new_design_pts

(Input(np.float64(0.4958715961234705), np.float64(2.5514364121459465e-08)),
 Input(np.float64(0.9999966883166532), np.float64(0.5447807986743418)),
 Input(np.float64(3.581779252082029e-07), np.float64(0.3680394329137556)),
 Input(np.float64(0.6047052623895744), np.float64(1.9163934035504315e-05)),
 Input(np.float64(0.2681917022018456), np.float64(0.9999953585128315)))

Note how the new design points all lie within the simulator domain we defined earlier,
i.e. they all lie in the unit square.

The final step is to update the GP using the newly-calculated design points. This first
requires us to compute the simulator values at the design points (in our case, using the
toy function defined earlier):

In [48]:
new_outputs = [simulation_func(x) for x in new_design_pts]

To update the GP, we combine the old training data with the new input/outputs calculated:

In [49]:
new_data = [TrainingDatum(x, y) for x, y in zip(new_design_pts, new_outputs)]
training_data = list(gp.training_data) + new_data
gp.fit(training_data)

# Sense-check that the updated GP now has the combined data
assert(len(gp.training_data) == len(initial_data) + len(new_data))

## Using a different GP for leave-one-out sampling methodology

By default, the leave-one-out errors GP calculated during the adaptive sampling method
uses a fresh copy of the supplied GP. In particular, it will use the same kernel function
as the original. We can instead specify that a different GP is used by supplying a new one
with the settings we desire. For example, to ensure that the leave-one-out errors GP uses
a squared exponential kernel instead of a Matern 5/2, we can do the following:

In [50]:
sqexp_gp = MogpEmulator(kernel="SquaredExponential")
new_design_pts2 = compute_single_level_loo_samples(gp, domain, batch_size=2, loo_errors_gp=sqexp_gp)
new_design_pts2

Too few unique inputs; defaulting to flat priors
Too few unique inputs; defaulting to flat priors


(Input(np.float64(0.7903157291293377), np.float64(0.7431890062662102)),
 Input(np.float64(0.7580092915972919), np.float64(0.9999999457736009)))